## Model XGBOOST and Endpoint for Deploy

## Imports

In [ ]:
# Imports
import os
import json
import sagemaker
import boto3
import numpy as np
import pandas as pd
from sagemaker.serializers import CSVSerializer
from sagemaker.inputs import TrainingInput
from sagemaker.predictor import Predictor
from sagemaker import get_execution_role

## Carrega os Dados

In [ ]:
# Obtém a sessão do SageMaker
session = boto3.Session()

In [ ]:
s3 = session.resource('s3')

In [ ]:
s3

In [ ]:
from sagemaker import get_execution_role
role = get_execution_role()
print(role)

In [ ]:
# Altere para o nome do seu bucket
s3_bucket = 'eduardo-project-medical-data'
prefix = 'dados'

In [ ]:
raiz = 's3://{}/{}/'.format(s3_bucket, prefix)
print(raiz)

In [ ]:
dados_treino = TrainingInput(s3_data = raiz + 'treino.csv', content_type = 'csv')
dados_teste = TrainingInput(s3_data = raiz + 'teste.csv', content_type = 'csv')

In [ ]:
print(json.dumps(dados_treino.__dict__, indent = 2))

In [ ]:
print(json.dumps(dados_teste.__dict__, indent = 2))

## Construção e Treinamento do Modelo XGB

In [ ]:

container_uri = sagemaker.image_uris.retrieve(region = session.region_name,
                                              framework = 'xgboost',
                                              version = '1.0-1',
                                              image_scope = 'training')

Criação do Container - seguindo a documentação da AWS estou criando um container para podermos usar apenas para treinamento uma máquina mais
potente disponível na versão gratuita do SageMaker. Para os parametros da função abaixo estou recuperando os dados da região utilizando o BOTO, os demais
parametros são ajustaveis, mas é necessário conferir os custos.

In [ ]:
# Argumentos do estimador para serem usados na função de criação
sagemaker_execution_role = role
sagemaker_session = sagemaker.Session()

In [ ]:
# Criação do Estimador - estou seguindo a documentação para a criação.
xgb = sagemaker.estimator.Estimator(image_uri = container_uri,
                                    role = sagemaker_execution_role,
                                    instance_count = 2,
                                    instance_type = 'ml.m5.large', #note que essa máquina/instancia tem apenas 50 hrs no nível gratuito.
                                    output_path = 's3://{}/artefatos'.format(s3_bucket),
                                    sagemaker_session = sagemaker_session,
                                    base_job_name = 'classifier')

In [ ]:
# Definição dos Hiperparâmetros - consultar a documenação caso queira mudar
xgb.set_hyperparameters(objective = 'binary:logistic', num_round = 100)

In [ ]:
# Treinamento do modelo
xgb.fit({'train': dados_treino, 'validation': dados_teste})

## Gerando o Endpoint a Partir do Modelo

In [ ]:

xgb_predictor = xgb.deploy(initial_instance_count = 2, instance_type = 'ml.m5.large')

Deploy do modelo treinado criando o endpoint o .deploy ajusta o resultado para que possa ser usado os resultados para outros propositos, inclusive o deploy
e dessa forma salvamos o modelo para ser usado por aplicações.

## Previsões a Partir do Endpoint

In [ ]:
csv_serializer = CSVSerializer()

In [ ]:
predictor = Predictor(endpoint_name = xgb_predictor.endpoint_name, serializer = csv_serializer)

In [ ]:
df_teste = pd.read_csv(raiz + 'teste.csv', names = ['class', 'bmi', 'diastolic_bp_change', 'systolic_bp_change', 'respiratory_rate'])

In [ ]:
df_teste.head()

,class,bmi,diastolic_bp_change,systolic_bp_change,respiratory_rate
0,0,-0.940089,-0.403964,-0.279542,-0.817379
1,0,-0.502614,-0.665582,0.131742,-0.362450
2,0,1.078473,0.347981,0.228029,-0.817379
3,1,-0.636164,-0.251491,0.587034,-0.817379
4,1,-0.528479,2.037253,1.383463,0.185934


In [ ]:
X = df_teste.sample(1)
X

,class,bmi,diastolic_bp_change,systolic_bp_change,respiratory_rate
1017,0,-0.022864,-0.496655,2.153753,-0.067314


In [ ]:
X = X.values[0]
X[1:]

array([-0.02286428, -0.49665455,  2.15375335, -0.06731361])

In [ ]:
paciente = X[1:]
paciente

array([-0.02286428, -0.49665455,  2.15375335, -0.06731361])

In [ ]:
# Faz a previsão de um paciente
predicted_class_prob = predictor.predict(paciente).decode('utf-8')
if float(predicted_class_prob) < 0.5:
    print('Previsão = Não Diabético')
else:
    print('Previsão = Diabético')
print()

Previsão = Não Diabético



## Avaliando o Modelo

In [ ]:
# Previsão de todos os pacientes no dataset de teste
predictions = []
expected = []
correct = 0
for row in df_teste.values:
    expected_class = row[0]
    payload = row[1:]
    predicted_class_prob = predictor.predict(payload).decode('utf-8')
    predicted_class = 1
    if float(predicted_class_prob) < 0.5:
        predicted_class = 0
    if predicted_class == expected_class:
        correct += 1
    predictions.append(predicted_class)
    expected.append(expected_class)

In [ ]:
print('Acurácia = {:.2f}%'.format(correct/len(predictions) * 100))

Acurácia = 77.72%


#### Confusion Matrix

In [ ]:
expected = pd.Series(np.array(expected))
predictions = pd.Series(np.array(predictions))
pd.crosstab(expected, predictions, rownames = ['Actual'], colnames = ['Predicted'], margins = True)

Predicted,0,1,All
Actual,,,
0.0,1909,71,1980
1.0,483,24,507
All,2392,95,2487


**That's all folks**

In [2]:
!jupyter nbconvert --to markdown Model-XGBoost-Training-and-Endpoint.ipynb

[NbConvertApp] WARNING | pattern 'Model-XGBoost-Training-and-Endpoint.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer